In [6]:
import numpy as np
import matplotlib.pyplot as plt
import mpld3 # interactive matplotlib
import pydot # interface to graphviz --> penso per visualizzare i diagrammi

from IPython.display import SVG, display
, 
from pydrake.geometry import MeshcatVisualizer, StartMeshcat, Rgba
from pydrake.math import RotationMatrix, RigidTransform
from pydrake.common.eigen_geometry import AngleAxis

mpld3.enable_notebook()



from pydrake.systems.framework import DiagramBuilder


from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser
from pydrake.multibody.meshcat import JointSliders

from pydrake.trajectories import PiecewisePose

In [7]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [8]:
# definition of frame for our task ---> store all in a dict
def MakeGripperFrames(X_gripper:dict, X_obj:dict): 
    '''
    input: original position of gripper, original position of obj
    All is a dict of rigid transform ---> rotation + translation 
    out: complete set of transformation needed for the task, + times dict
    
    '''
    
    p_Ggrasp_OBJ = [0, 0.11, 0]
    R_Ggrasp_OBJ = (
        RotationMatrix.MakeXRotation(np.pi/2.0) @ 
        RotationMatrix.MakeZRotation(np.pi/2.0)
    )
    X_Ggrasp_OBJ = RigidTransform(R_Ggrasp_OBJ, p_Ggrasp_OBJ)
    X_Ggrasp_Gpregrasp = RigidTransform([0, -0.08, 0]) 
    
    
    X_gripper['pick'] = X_obj['initial'] @ X_Ggrasp_OBJ.inverse()
    X_gripper['place'] = X_obj['goal'] @ X_Ggrasp_OBJ.inverse()
    
    X_gripper['prepick'] = X_gripper['pick'] @ X_Ggrasp_Gpregrasp
    X_gripper['preplace'] = X_gripper['place'] @ X_Ggrasp_Gpregrasp
    
    # interpolate orientation by convertin to axis-angle: idk---> see again the book
    # MAKE ROTATION OF FIXED ANGLE ALONG FIXED VECTOR  
    # angle_axis : rotate around axis AXS of angle ALPHA
    # -----------------------------------
    X_Ginitial_Gprepick = X_gripper['initial'].inverse() @ X_gripper['prepick']
    X_Gprepick_Gpreplace = X_gripper['prepick'].inverse() @ X_gripper['preplace']
    angle_axis = X_Gprepick_Gpreplace.rotation().ToAngleAxis()
    # DONT UNDERSTAND WHY THIS CALC, and i mean what is the use of ANGLE AXIS
    X_Gprepick_Gclearance = RigidTransform(
        AngleAxis(angle=angle_axis.angle() / 2.0, axis=angle_axis.axis()), 
        X_Gprepick_Gpreplace.translation() / 2.0 + np.array([0, -0.3, 0])
    
    )
    X_gripper['clearance'] = X_G['prepick'] @ X_Gprepick_Gclearance
    
    #---------------------------------------------------------------
    #---------------------------------------------------------------
    #---------------------------------------------------------------
    # SET TIMING: 
    times = {'initial':0}
    times['prepick'] = times['initial'] + 10.0 * np.linalg.norm(X_Ginitial_Gprepick.translation())
    # adding the norm means say that the time is proportional to the distance with a factor 10

    times['pick_start'] = times['prepick'] + 2.0
    times['pick_end'] = times['pick_start'] + 2.0
    X_gripper['pick_start'] = X_gripper['pick']
    X_gripper['pick_end'] = X_gripper['pick']
    
    times['postpick'] = times['pick_end'] + 2.0
    X_gripper['postpick'] = X_gripper['prepick']
    
    
    time_to_from_clearance = 10.0 * np.linalg.norm(X_Gprepick_Gclearance.translation())
    times["clearance"] = times["postpick"] + time_to_from_clearance
    times["preplace"] = times["clearance"] + time_to_from_clearance
    times["place_start"] = times["preplace"] + 2.0
    times["place_end"] = times["place_start"] + 2.0
    
    X_gripper['place_start'] = X_gripper['place']
    X_gripper['place_end'] = X_gripper['place']
    
    times['postplace'] = times['place_end'] + 2.0
    X_gripper['postplace'] = X_gripper['preplace']
    
    return X_gripper, times


X_G = {
    "initial": RigidTransform(
        RotationMatrix.MakeXRotation(-np.pi / 2.0), [0, -0.25, 0.25]
    )
}
X_O = {
    "initial": RigidTransform(
        RotationMatrix.MakeZRotation(np.pi / 2.0), [-0.2, -0.75, 0.025]
    ),
    "goal": RigidTransform(
        RotationMatrix.MakeZRotation(np.pi), [0.75, 0, 0.025]
    ),
}
X_G, times = MakeGripperFrames(X_G, X_O)
print(
    f"Sanity check: The entire maneuver will take {times['postplace']} seconds to execute."
)

Sanity check: The entire maneuver will take 30.905782746839105 seconds to execute.


In [9]:
X_G.keys()

dict_keys(['initial', 'pick', 'place', 'prepick', 'preplace', 'clearance', 'pick_start', 'pick_end', 'postpick', 'place_start', 'place_end', 'postplace'])

In [10]:
#just for visualizer the pose that we get

def visualize_gripper_frames(X_G, X_O):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    parser = Parser(plant, scene_graph)
    parser.SetAutoRenaming(True)
    for key, pose in X_G.items():
        g = parser.AddModelsFromUrl(
            "package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"
        )[0]
        plant.WeldFrames(
            plant.world_frame(), plant.GetFrameByName("body", g), pose
        )
    for key, pose in X_O.items():
        o = parser.AddModelsFromUrl(
            "package://drake/examples/manipulation_station/models/061_foam_brick.sdf"
        )[0]
        plant.WeldFrames(
            plant.world_frame(), plant.GetFrameByName("base_link", o), pose
        )

    plant.Finalize()

    meshcat.Delete()
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.ForcedPublish(context)


visualize_gripper_frames(X_G, X_O)

dict_keys(['initial', 'prepick', 'preplace', 'clearance', 'pick_start', 'pick_end', 'postpick', 'place_start', 'place_end', 'postplace'])

In [25]:
# X_G.pop('pick') ---> same of pre_pick
# X_G.pop('place') ---> same of pre_place
trajetory = PiecewisePose.MakeLinear(list(times.values()), list(X_G.values()))



In [37]:
trajectory_path_GRIPPER = trajetory.get_position_trajectory()
pos_gripper = trajectory_path_GRIPPER.vector_values(trajectory_path_GRIPPER.get_segment_times())


plt.plot(trajectory_path_GRIPPER.get_segment_times(), pos_gripper.T)
plt.legend(['x', 'y', 'z'])
plt.title('pos_gripper')
mpld3.display()

In [42]:
meshcat.ResetRenderMode()
meshcat.SetLine('pos_gripper', pos_gripper, 5.0, rgba=Rgba(1, 0.65, 0))